# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Generate-pre-processed-FASTA-files" data-toc-modified-id="Generate-pre-processed-FASTA-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate pre-processed FASTA files</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-low-spike" data-toc-modified-id="Prepare-low-spike-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Prepare low-spike</a></div><div class="lev2 toc-item"><a href="#Prepare-high-spikes" data-toc-modified-id="Prepare-high-spikes-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Prepare high-spikes</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Simulate MS-data</a></div><div class="lev1 toc-item"><a href="#Post-process-data" data-toc-modified-id="Post-process-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Post-process data</a></div>

# Generate pre-processed FASTA files

Using the custom-made `seqtk generate_spikein.py` script.

Dependencies:

* OpenMS binaries in PATH (version 2.2.0)

## Setup

In [21]:
threads=6
run=batch9_synced
mkdir ${run}

In [22]:
sample_names_low="l1 l2 l3"
sample_names_high="h1 h2 h3"
samples="${sample_names_low} ${sample_names_high}"

In [23]:
echo -e "name\tbiorepgroup\ttechrepgroup\tcondition" > ${run}/design.tsv
echo -e "l1\t1\t1\tlow" >> ${run}/design.tsv
echo -e "l2\t2\t1\tlow" >> ${run}/design.tsv
echo -e "l3\t3\t1\tlow" >> ${run}/design.tsv
echo -e "h1\t1\t1\thigh" >> ${run}/design.tsv
echo -e "h2\t2\t1\thigh" >> ${run}/design.tsv
echo -e "h3\t3\t1\thigh" >> ${run}/design.tsv

cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition
l1	1	1	low
l2	2	1	low
l3	3	1	low
h1	1	1	high
h2	2	1	high
h3	3	1	high


## Prepare low-spike

In [30]:
lfqtk generate_spikein_set \
    --background_fa data/uniprot_ecoli.pure.fasta \
    --spikein_fa data/uniprot_potato.pure.fasta \
    --offset_mean 0 \
    --offset_std 0 \
    --base_int 1000000 \
    --noise_std 0 \
    --back_count 20 \
    --spike_count 5 \
    --spike_folds "1,1,1,8,8,8" \
    --out_base ${run} \
    --sample_names "l1.fa,l2.fa,l3.fa,h1.fa,h2.fa,h3.fa" \
    --verbose

4436 entries loaded from data/uniprot_ecoli.pure.fasta as background
403 entries loaded from data/uniprot_potato.pure.fasta as spike-in
20 entries picked as background, 5 as spike-in
25 entries written to batch9_synced/l1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch9_synced/l2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch9_synced/l3.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch9_synced/h1.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch9_synced/h2.fa
20 entries picked as background, 5 as spike-in
25 entries written to batch9_synced/h3.fa


In [25]:
# for name in ${sample_names_low}; do
#     echo "Generating sample: ${name}"
#     lfqtk generate_spikein \
#         --background_fa data/uniprot_ecoli.pure.fasta \
#         --spikein_fa data/uniprot_potato.pure.fasta \
#         --output_fa ${run}/${name}.fa \
#         --offset_mean 0 \
#         --offset_std 0 \
#         --back_int 1000000 \
#         --back_noise_std 0 \
#         --back_count 20 \
#         --spike_int 1000000 \
#         --spike_noise_std 0 \
#         --spike_count 5 \
#         --verbose
# done

## Prepare high-spikes

In [28]:
# for name in ${sample_names_high}; do
#     echo "Generating sample: ${name}"
#     lfqtk generate_spikein \
#         --background_fa data/uniprot_ecoli.pure.fasta \
#         --spikein_fa data/uniprot_potato.pure.fasta \
#         --output_fa ${run}/${name}.fa \
#         --offset_mean 0 \
#         --offset_std 0 \
#         --back_int 1000000 \
#         --back_noise_std 0 \
#         --back_count 20 \
#         --spike_int 8000000 \
#         --spike_noise_std 0 \
#         --spike_count 5 \
#         --verbose
# done

# Simulate MS-data

Generate the OpenMS tool `MSSimulator`.

In [33]:
time parallel -j ${threads} \
"echo \"Processing {}\"
MSSimulator \
    -in ${run}/{}.fa \
    -out_fm ${run}/{}.ground.featureXML \
    -out_id ${run}/{}.ground.idXML \
    -out ${run}/{}.mzML \
    -out_pm ${run}/{}.centroided.mzML" \
    ::: ${samples}


Processing l3
Loading sequence data from batch9_synced/l3.fa ...
done (25 protein(s) loaded)
Starting simulation
2017/11/06, 11:52:20: Digest Simulation ... started
2017/11/06, 11:52:20: RT Simulation ... started
2017/11/06, 11:52:22: Predicting RT ... done
2017/11/06, 11:52:22: RT prediction gave 'invalid' results for 742 peptide(s), making them unobservable.
2017/11/06, 11:52:22:   (List is too big to show)
2017/11/06, 11:52:22: Creating experiment with #501 scans ... done
2017/11/06, 11:52:22: Detectability Simulation ... started
2017/11/06, 11:52:22: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/06, 11:52:22: Simulating 834 features
Progress of 'Ionization':
-- done [took 4.52 s (CPU), 4.53 s (Wall)] -- 
2017/11/06, 11:52:26: #Peptides not ionized: 0
2017/11/06, 11:52:26: #Peptides outside mz range: 275
2017/11/06, 11:52:26: Raw MS1 Simulation ... started
2017/11/06, 11:52:26:   Simulating signal for 2319 features ...
Progress of 'RawMSS

Loading sequence data from batch9_synced/h1.fa ...
done (25 protein(s) loaded)
Starting simulation
2017/11/06, 11:52:20: Digest Simulation ... started
2017/11/06, 11:52:20: RT Simulation ... started
2017/11/06, 11:52:21: Predicting RT ... done
2017/11/06, 11:52:21: RT prediction gave 'invalid' results for 742 peptide(s), making them unobservable.
2017/11/06, 11:52:21:   (List is too big to show)
2017/11/06, 11:52:21: Creating experiment with #501 scans ... done
2017/11/06, 11:52:21: Detectability Simulation ... started
2017/11/06, 11:52:21: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/06, 11:52:21: Simulating 834 features
Progress of 'Ionization':
-- done [took 5.88 s (CPU), 5.90 s (Wall)] -- 
2017/11/06, 11:52:27: #Peptides not ionized: 0
2017/11/06, 11:52:27: #Peptides outside mz range: 275
2017/11/06, 11:52:27: Raw MS1 Simulation ... started
2017/11/06, 11:52:27:   Simulating signal for 2319 features ...
Progress of 'RawMSSignal':
-- don

In [34]:
# for xml in *.featureXML; do 
#     ProteinQuantifier \
#         -in ${xml} \
#         -peptide_out ${xml%.*}.csv
# done

In [35]:
time parallel -j ${threads} \
"echo \"Processing {}\"
FeatureFinderCentroided \
    -in ${run}/{}.centroided.mzML \
    -out ${run}/{}.featureXML" \
    ::: ${samples}


Processing h3
Progress of 'loading spectra list':
-- done [took 0.06 s (CPU), 0.06 s (Wall)] -- 
Progress of 'Precalculating intensity scores':
-- done [took 0.05 s (CPU), 0.05 s (Wall)] -- 
Progress of 'Precalculating mass trace scores':
-- done [took 0.12 s (CPU), 0.12 s (Wall)] -- 
Progress of 'Precalculating isotope distributions':
-- done [took 0.02 s (CPU), 0.01 s (Wall)] -- 
Progress of 'Calculating isotope pattern scores for charge 1':
-- done [took 0.20 s (CPU), 0.21 s (Wall)] -- 
Progress of 'Finding seeds for charge 1':
-- done [took 0.02 s (CPU), 0.02 s (Wall)] -- 
Found 15197 seeds for charge 1.
Progress of 'Extending seeds for charge 1':
-- done [took 5.30 s (CPU), 5.31 s (Wall)] -- 
Found 3371 feature candidates for charge 1.
Progress of 'Calculating isotope pattern scores for charge 2':
-- done [took 0.26 s (CPU), 0.25 s (Wall)] -- 
Progress of 'Finding seeds for charge 2':
-- done [took 0.01 s (CPU), 0.02 s (Wall)] -- 
Found 7086 seeds for charge 2.
Progress of 'Extend

-- done [took 0.29 s (CPU), 0.28 s (Wall)] -- 
Progress of 'Finding seeds for charge 2':
-- done [took 0.01 s (CPU), 0.02 s (Wall)] -- 
Found 7086 seeds for charge 2.
Progress of 'Extending seeds for charge 2':
-- done [took 3.78 s (CPU), 3.78 s (Wall)] -- 
Found 1207 feature candidates for charge 2.
Progress of 'Calculating isotope pattern scores for charge 3':
-- done [took 0.26 s (CPU), 0.26 s (Wall)] -- 
Progress of 'Finding seeds for charge 3':
-- done [took 0.01 s (CPU), 0.01 s (Wall)] -- 
Found 4374 seeds for charge 3.
Progress of 'Extending seeds for charge 3':
-- done [took 2.08 s (CPU), 2.08 s (Wall)] -- 
Found 594 feature candidates for charge 3.
Progress of 'Calculating isotope pattern scores for charge 4':
-- done [took 0.44 s (CPU), 0.45 s (Wall)] -- 
Progress of 'Finding seeds for charge 4':
-- done [took 0.01 s (CPU), 0.01 s (Wall)] -- 
Found 1787 seeds for charge 4.
Progress of 'Extending seeds for charge 4':
-- done [took 1.01 s (CPU), 1.02 s (Wall)] -- 
Found 207 fea

In [36]:
for sample in ${samples}; do \
    echo "Processing sample: ${sample}"
    IDMapper \
        -id ${run}/${sample}.ground.idXML \
        -in ${run}/${sample}.featureXML \
        -out ${run}/${sample}.mapped.featureXML \
        > ${run}/${sample}.mapped.featureXML.log
done

Processing sample: l1
Processing sample: l2
Processing sample: l3
Processing sample: h1
Processing sample: h2
Processing sample: h3


Here, we do RT alignment (?) of features (with their identities mapped).

In [ ]:
# out_strings=""
# for sample in ${samples}; do
#     out_strings="${out_strings} ${run}/${sample}.mapped.aligned.featureXML"
# done

# echo ${out_strings}

# MapAlignerPoseClustering \
#     -in ${run}/*.mapped.featureXML \
#     -out ${out_strings}


In [37]:
# FeatureLinkerUnlabeledQT \
#     -in ${run}/*.mapped.aligned.featureXML \
#     -out ${run}/combined.consensusXML
    
FeatureLinkerUnlabeledQT \
    -in ${run}/*.mapped.featureXML \
    -out ${run}/combined.consensusXML

Progress of 'reading input':
-- done [took 1.20 s (CPU), 1.21 s (Wall)] -- 
Progress of 'linking features':
-- done [took 0.44 s (CPU), 0.44 s (Wall)] -- 
Number of consensus features:
  of size  6:   2342
  total:        2342
FeatureLinkerUnlabeledQT took 2.30 s (wall), 2.28 s (CPU), 0.00 s (system), 2.28 s (user).


# Post-process data

Extract and prepare the consensus data for normalization - transform it to an appropriate format.

In [38]:
TextExporter \
    -in ${run}/combined.consensusXML \
    -out ${run}/combined.linked_features.csv \
    -consensus:features ${run}/combined.features.csv \
    -consensus:elements ${run}/combined.elements.csv \
    -consensus:centroids ${run}/combined.centroids.csv

Warning while exporting 'batch9_synced/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch9_synced/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch9_synced/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch9_synced/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
Warning while exporting 'batch9_synced/combined.consensusXML': protein identification run ID '_0000-00-00T00:00:00' occurs more than once
TextExporter took 0.74 s (wall), 0.73 s (CPU), 0.00 s (system), 0.73 s (user).


Not sure why, but the `combined.features.csv` file included extra empty fields with numbers beyond the actual sample range. These are omitted here, but should probably be remedied upstreams instead.

In [ ]:
s_count=$(echo ${samples} | tr " " "\n" | wc -l)
cut_end=$(echo "${s_count} * 5 + 9" | bc)
cut -f 1-${cut_end} ${run}/combined.features.csv \
    > ${run}/combined.features.sub.csv

In [ ]:
util_scripts/openms_to_normalyzer.py \
    -i ${run}/combined.features.sub.csv \
    -o ${run}/combined.final.tsv \
    --design ${run}/design.tsv